In [14]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from env import github_token, github_username
import acquire as aqr
import prepare as prep
import explore as exp
import json

In [2]:
# Specify the file name where you want to save the list
file_name = "repo_processed.json"

# Load the REPOS list from the JSON file
with open(file_name, 'r') as file:
    repo_loaded = json.load(file)

In [3]:
repos_df = pd.DataFrame(repo_loaded)
repos_df

,repo,language,readme_contents
0,AtsushiSakai/PythonRobotics,Python,"<img src=""https://github.com/AtsushiSakai/Pyth..."
1,kiloreux/awesome-robotics,None,Awesome Robotics\n================\n\n[![Aweso...
2,NxRLab/ModernRobotics,Python,"# Modern Robotics: Mechanics, Planning, and C..."
3,mithi/robotics-coursework,None,# [🐳](https://mithi.github.io/deep-blueberry) ...
4,onlytailei/CppRobotics,C++,# CppRobotics\n\nThis is the cpp implementatio...
...,...,...,...
672,CatchZeng/dingtalk,Go,# dingtalk\n\n![Go](https://github.com/CatchZe...
673,1c7/Translate-Subtitle-File,None,## 字幕组机翻小助手 - [视频演示](https://www.bilibili.com/...
674,landy22granatt/Kumpulan-Script-Termux,Python,pkg update && pkg upgrade\npkg install bash\np...
675,cirosantilli/china-dictatorship,HTML,"<style>\npre{ white-space:pre }\n#header,#cont..."


In [ ]:
extra_words = []
exclude_words = []
repos_df = prep.process_dataframe(repos_df, extra_words, exclude_words)

In [ ]:
repos_df = repos_df.drop(columns=['original', 'clean', 'remove_stopwords', 'stemmed'])

In [51]:
repos_df.head()

,repo,language,lemmatized
0,AtsushiSakai/PythonRobotics,Python,img srchttpsgithubcomatsushisakaipythonrobotic...
2,NxRLab/ModernRobotics,Python,modern robotics mechanic planning control code...
4,onlytailei/CppRobotics,C++,cpprobotics cpp implementation pythonroboticsh...
5,JdeRobot/RoboticsAcademy,JavaScript,hrefhttpsjderobotgithubioimg srcimglogogif wid...
6,pptacher/probabilistic_robotics,C++,probabilisticrobotics working detailed solutio...


In [50]:
# Remove rows where 'language' is None
repos_df = repos_df[repos_df['language'].notna()]

In [ ]:
repos_df[repos_df['language'].isna()]

,repo,language,lemmatized


In [ ]:
repos_df['lemmatized'] == 

In [53]:
counts = exp.show_counts_and_ratios(repos_df, "language")
counts

,n,percent
language,,
C++,207,0.336585
Python,202,0.328455
JavaScript,35,0.056911
C,19,0.030894
Jupyter Notebook,19,0.030894
Java,14,0.022764
TypeScript,14,0.022764
C#,13,0.021138
HTML,12,0.019512


In [43]:
counts['n'].sum()

615

In [44]:
len(counts)

36